<h2 align="center">CYFLOD</h2>

<h3 align="center">Date: 19 Jan 2024</h3>

<h3 align="center">Author: Nauman Ullah Gilal</h3>

<h3 align="center">Supervisor: Dr. Marco Agus</h3>

<h3 align="center">Institution: College of Science and Engineering,<br>Hamad Bin Khalifa University</h3>



# Declaration of Efficient Net family

In [ ]:
architecture = 'efficientnet-b4'
logfile = 'efficientnet-b0-MENA.csv'

## The tables
effnet_size = ({
    'efficientnet-b0':300,      # original resolution
    'efficientnet-b1':240,      # original resolution
    'efficientnet-b2':260,      # original resolution
    'efficientnet-b3':400,      # original resolution
    'efficientnet-b4':500,      # original resolution
    'efficientnet-b5':456,      # original resolution
    'efficientnet-b6':528,      # original resolution
    'efficientnet-b7':600,      # original resolution
    'efficientnet-lite0':224,
    'efficientnet-lite2':260,
    'efficientnet-lite4':384,
    'resnet18':224,
    'resnet34':224,
    'resnet50':224,
    'resnet101':224,
    'resnet152':224
})
batch_size = ({ 
    'efficientnet-b0':128, # bs = 256 for B1
    'efficientnet-b1':120,      #
    'efficientnet-b2':160,       #88 160
    'efficientnet-b3':16,       #
    'efficientnet-b4':32,       #
    'efficientnet-b5':14,       #
    'efficientnet-b6':16,
    'efficientnet-b7':8,
    'efficientnet-lite0': 160,
    'efficientnet-lite2': 160,
    'efficientnet-lite4':24,
    
    'resnet18':400,             # 3080: ok
    'resnet34':320,            # RTX 3090 rocz!
    'resnet50':256,             # 3080: ok
    'resnet101':96,
    'resnet152':64
})
learning_rate = ({
    'efficientnet-b0':5e-4,     # 2e-3: too large, 5e-4: too large
    'efficientnet-b1':2e-3,     # 1e-3: 82.53/93.9
    'efficientnet-b2':1e-3,     # 1e-3, 2e-3     
    'efficientnet-b3':2e-3,    
    'efficientnet-b4':2e-3,  
    'efficientnet-b5':1.5e-3,  
    'efficientnet-b6':4e-3,
    'efficientnet-b7':4e-3,
    'efficientnet-lite0': 5e-4,
    'efficientnet-lite2':1e-3,
    'efficientnet-lite4':5e-4,
    'resnet18':2.823e-02,
    'resnet34':-1,
    'resnet50':1.016e-02,
    'resnet101':-1,
    'resnet152':-1,
})
epochs = ({
    'efficientnet-b0':(1,100), # for initally top losses
    'efficientnet-b1':(6,20),
    'efficientnet-b2':(15,50),
    'efficientnet-b4':(5,25),
    'efficientnet-lite0': (15,80),
    'efficientnet-lite2': (5,25),
    'resnet18':(10,50),
    'resnet34':(10,50),
    'resnet50':(10,50),
    'resnet101':(10,50),
    'resnet152':(10,50)
})
label_smoothing = ({           # a = 0.00     a = 0.05     a = 0.10     a = 0.15     a = 0.20
    'efficientnet-b0':0.05,    #
    'efficientnet-b1':0.05,    #
    'efficientnet-b2':0.05,    # 0.05
    'efficientnet-b3':0.05,    #
    'efficientnet-b4':0.05,    #
    'efficientnet-b5':0.05,
    'efficientnet-b6':0.05,
    'efficientnet-b7':0.05,
    'efficientnet-lite0': 0.05,
    'efficientnet-lite2': 0.05,
    'resnet18':0.05,
    'resnet34':0.05,
    'resnet50':0.05,
    'resnet101':0.05,
    'resnet152':0.05
})
## Testing accuracies (top1,top5)
scores = ({
    'efficientnet-b0':(0,0),
    'efficientnet-b1':(0,0),
    'efficientnet-b2':(0,0),
    'efficientnet-b3':(0,0),
    'efficientnet-b4':(0,0),
    'efficientnet-lite0':(0,0),
    'efficientnet-lite2': (0,0),
    'resnet50':(0,0)
})
tta_scores = ({
    'efficientnet-b0':(0,0),
    'efficientnet-b1':(0,0),
    'efficientnet-b2':(0,0),
    'efficientnet-b3':(0,0),
    'efficientnet-b4':(0,0),
    'efficientnet-lite0':(0,0),
    'efficientnet-lite2': (0,0),
    'efficientnet-lite4':(0,0),
    'resnet50':(0,0)
})

 # Writing training data into csv

In [ ]:
import pandas as pd
train_df = pd.read_csv(dataset+"_"+noise_type+"_"+str(noise_ratio)+".csv")
train_df

In [ ]:
print(train_df.Noisy_labels.tolist())

# Viualization of Training Data, Samples per class

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.facecolor'] = 'white'
lbl = train_df['Label_numeric']
# print(lbl)
# print(lbl.size)
plt.figure(figsize=(100,20))
ax = sns.countplot(x= lbl, data=train_df)
ax.bar_label(ax.containers[0], weight = 'bold', fontsize = '30', color = 'red')
plt.xticks(rotation=90, fontsize = 40, weight = 'bold', color = 'black')
plt.yticks( fontsize = 40, weight = 'bold', color = 'black')
plt.xlabel("Labels", fontsize = 40, weight = 'bold', color = 'black')
plt.ylabel("Number of Images", fontsize = 40, weight = 'bold', color = 'black')

# Images Per Catergory

In [ ]:
number_classes = train_df['Label'].nunique()
print("Number of classes:", number_classes)
counted = train_df.groupby(["Label"]).size()
print(counted)

# Writing testing  Data into CSV  (Test data frame df)

In [ ]:
from pathlib import Path
import os.path
test_dir = "testing"
test_path = Path(test_dir)
filepaths = list(test_path.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
test_df = pd.DataFrame(images)
test_df

In [ ]:
test_df['Label'] = test_df['Label'].map(lambda x: x.lower())

In [ ]:
sorted_df = test_df.sort_values(by='Label').reset_index(drop=True)
sorted_df

In [ ]:
my_list = sorted_df.Label.unique()

In [ ]:
my_dict = {value: index for index, value in enumerate(my_list)}

In [ ]:
sorted_df['Label_numeric'] = sorted_df['Label'].map(my_dict)
sorted_df

In [ ]:
test_df = sorted_df[['Filepath', 'Label_numeric', 'Label']]
test_df

In [ ]:
label_counts = sorted_df['Label_numeric'].value_counts().sort_index().tolist()
print(label_counts)

In [ ]:
labels = sorted_df.Label_numeric.tolist()
print(labels)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
lbl = test_df['Label_numeric']
# print(lbl)
# print(lbl.size)
plt.figure(figsize=(70,20))
ax = sns.countplot(x= lbl, data=test_df)
ax.bar_label(ax.containers[0], weight = 'bold', fontsize = '30', color = 'red')
plt.xticks(rotation=90, fontsize = 40, weight = 'bold', color = 'black')
plt.yticks( fontsize = 40, weight = 'bold', color = 'black')
plt.xlabel("Labels", fontsize = 40, weight = 'bold', color = 'black')
plt.ylabel("Number of Images", fontsize = 40, weight = 'bold', color = 'black')

# Import Fastai and pytorch Library

In [ ]:
from fastai.vision.all import *
from fastai import __version__
import torch
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
from fastai.vision import *
from fastai import optimizer, losses, metrics
from functools import partial, wraps
import matplotlib.pyplot as plt
import random

# Fastai version checking

In [ ]:
import fastai
import torchvision
print(fastai.__version__)
print(torch.__version__)
print(torchvision.__version__)

# Torch version checking

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)
torch.cuda.is_available()


# Preparation of data for learner (Fastai learner)

In [ ]:
batch_tfms = [Zoom(),Rotate(), Flip(), Brightness(), Contrast(), Saturation()]

In [ ]:

# default augmentation
bs   = batch_size[architecture]
imgs = effnet_size[architecture]
resize = (imgs*4)//3
data0 = (ImageDataLoaders.from_df(train_df, valid_pct=0.2,bs=bs,label_col=1,
                                        shuffle_train=True,
                                        item_tfms=Resize(imgs),
                                        batch_tfms = batch_tfms))

print("Image size=", imgs)
print("Batch size=", bs)
print("Architecture=", architecture)
print(resize)

In [ ]:
data0.show_batch()

In [ ]:
import damped_losses as losses
loss_func = losses.DumpledSCELossFlat(num_classes=196, alpha = 0.1, beta = 1.0, delta=0.0, reduction='mean')
model = EfficientNet.from_pretrained(architecture, num_classes=196)
learner_type = Learner

In [ ]:
best_pth = best_pth
checkpoints = SaveModelCallback(fname=best_pth,monitor='accuracy',comp=np.greater, with_opt=True)
# brainfreeze = BnFreeze()
learn = ( learner_type(data0, model,metrics=[accuracy],loss_func=loss_func,
                        cbs=[ShowGraphCallback(),checkpoints]).to_fp16())
print("Best pth is=", best_pth)

In [ ]:
learn.lr_find()
learn.recorder.plot_lr_find()

In [ ]:
low_lr = 1e-4
lr = 1e-3
learn.fit_one_cycle(10, lr_max = slice(low_lr, lr))

# Unfreeze the Learner
fast.ai freezes the pre-trained part when we create the model. So up to this point, 
we only trained the last layer block. Next, we will unfreeze the pre-trained part and train the whole model.

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(50, lr_max = slice(low_lr, lr))

In [ ]:
data_test = (ImageDataLoaders.from_df(test_df, valid_pct = 0.0, splitter=None, shuffle=False, label_col=1, item_tfms=Resize(imgs)))
preds = learn.get_preds(dl=data_test)
preds
print("length of preds[1]",len(preds[1]))
acc= accuracy(preds[0], preds[1])
print(" BaselineTop-1 Accuracy:", acc)
# inter_test0 = ClassificationInterpretation.from_learner(tester0, dl =data_test)
# inter_test0.plot_confusion_matrix(figsize = (50,60))

 ------------
 B0|Epochs|20|8056   |
 -----------------------------

In [ ]:
err = {}
acc = []
log_preds, y  = learn.tta(dl=data_test)
tta_acc = accuracy(log_preds, y)
print(tta_acc)
err[0] = (100.0, 100.0*(1.0-float(tta_acc)))
acc.append(tta_acc)

In [ ]:
inter_test = ClassificationInterpretation.from_learner(learn, dl =data_test)
inter_test.plot_confusion_matrix(figsize = (50,50))

In [ ]:
PATH='models/'+feature_path+'.p'
torch.save(model.state_dict(), PATH)

In [ ]:
# import shutil
import os
# Specify the path of the folder where you want to create the directory
folder_path = 'models'

# Specify the name of the directory you want to create
directory_name = 'baseline'

# Combine the folder path and directory name
new_directory_path = os.path.join(folder_path, directory_name)

# Create the directory
try:
    # Create the directory
    os.makedirs(new_directory_path)
    print(f"Directory '{directory_name}' created inside '{folder_path}'.")
except FileExistsError:
    print(f"Directory '{directory_name}' already exists inside '{folder_path}'.")

# Now, here we copy the best model for baseline and past into the destination, like the folder we created inside the model directory, named "baseline"!!!!!
source = 'models/'+best_pth+'.pth'

destination = 'models/baseline/'+best_pth+'.pth'
# Copy the file from source to destination
shutil.copy(source, destination)

# Iterative Cleaning

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner
import random as rn
img_size = imgs
drop_idxx = []
los = []
top_losses = []
def class_frequencies(data,class_key='Label'):
    classes = set(data[class_key])
    class_freq = {}
    max_freq = 0.5
    min_freq =100000#1000000
    for cl in classes:
        class_df = data[data[class_key]==cl]
        freq = len(class_df)
        max_freq = max(freq,max_freq)
        min_freq = min(freq,min_freq)
        class_freq[cl] = freq
    print(f'Max frequency is {max_freq}')
    print(f'Min frequency is {min_freq}')
    return class_freq,max_freq,min_freq
    

def russian_roulette(idx, losses, data,class_key='Label', min_prob=0.5,max_prob=0.9): #min_prob=0.5,max_prob=0.9 original values
    
    class_freq,max_freq,min_freq = class_frequencies(data,class_key)
    min_prob = float(min_freq/max_freq)*max_prob
    print(f'Min probability = {min_prob}')
    deltap = (max_prob - min_prob)/(max_freq**2)

    drop_idx = []
    for n,i in enumerate(idx.numpy()):
        l = data.iloc[i][class_key]
        prob = min_prob + (class_freq[l]-min_freq)*deltap
        r = rn.random()
        if  r < prob:
            drop_idx.append(i)
            class_freq[l] -= 1 
    return drop_idx

def data_definition(data,img_size,tfms,model,best_pth,k_samples=100, min_prob=0.4, max_prob = 0.9, largest=True): # k_sample = 1000
    databunch = ( ImageDataLoaders.from_df(data, valid_pct=0.0, bs=bs,label_col=1,
                                        shuffle_train=True,
                                        item_tfms=Resize(imgs),
                                        batch_tfms = batch_tfms))
    learn_cln = (learner_type(databunch, model,metrics=[accuracy],loss_func=loss_func, cbs=[ShowGraphCallback(),checkpoints]).to_fp16())
    learn_cln.load(best_pth)
    entire_training_set_csv0 = (ImageDataLoaders.from_df(data, valid_pct = 0.0, splitter=None, shuffle=False, label_col=1, item_tfms=Resize(imgs)))
    interp = ClassificationInterpretation.from_learner(learn_cln, dl = entire_training_set_csv0)
    losses = None
    idx = None
    if hybrid:
        l_big,idx_big = interp.top_losses(k_samples//2,largest=True)
        l_sm,idx_sm = interp.top_losses(k_samples//2,largest=False)
        losses = torch.cat((l_big,l_sm),0)
        idx = torch.cat((idx_big,idx_sm),0)
    else:
        losses,idx = interp.top_losses(k_samples,largest=largest)
    drop_idx = russian_roulette(idx,losses,data,min_prob = min_prob, max_prob = max_prob)
    data_filtered = data.drop(data.index[drop_idx])
    print(f'Dropped {len(drop_idx)} top losses')
    return data_filtered

# Damped 

In [ ]:
drop_idxx = []
los = []
top_losses = []
n_training_series = 15  # orginally it is 15, but for testing purpose we are using with 1 to check the scheme with custom function.
min_epochs = 1
min_prob=0.5
max_prob=0.9
delta_epochs_per_serie =  0
min_lr = 1e-4
kappa_lr = 1 
k_samples = 0
data_per_serie = {}
data_per_serie[0] = train_df
largest = True
from_scratch = False
hybrid = False
cleaning_cycles = 3 
delta_max = 0.25
accs = []
deltas = []
for c in range(cleaning_cycles):
    for n in range(n_training_series):
        
        delta =  delta_max * math.sin( math.pi * float(n+1)/(n_training_series) )
        loss_func = losses.DumpledSCELossFlat(num_classes=196, alpha = 0.1, beta = 1.0, delta=delta, reduction='mean')
        k_samples = 50
        epochs_per_serie = min_epochs + n * delta_epochs_per_serie
        lr = min_lr * kappa_lr * (n_training_series - n)
        data_per_serie[n+1] = data_definition(data_per_serie[n],img_size,batch_tfms,model,best_pth,k_samples=k_samples, 
                                              min_prob = min_prob, max_prob=max_prob, largest = largest)
        print("************Round: ",n)
        print("************data per serie:******************")
        print("************length of data per serie:******************")
        print(len(data_per_serie[n+1]))
        databunch = ( ImageDataLoaders.from_df(data_per_serie[n+1], valid_pct = 0.2,bs=bs,label_col=1,
                                        shuffle_train=True,
                                        item_tfms=Resize(imgs),
                                        batch_tfms = batch_tfms))
        learn =(learner_type(databunch, model,metrics=[accuracy],loss_func=loss_func,
                            cbs=[ShowGraphCallback(),checkpoints]).to_fp16())
        learn.unfreeze() 
        learn.load(best_pth)
        learn.fit_one_cycle(epochs_per_serie, lr_max=slice(1e-6, 5e-5))
        if n%3== 0:
            log_preds, y = learn.tta(dl=data_test)
            print("************Round: ",n)
            tta_acc = accuracy(log_preds, y)
            print(f'TTA Accuracy:{tta_acc}')
            acc.append(tta_acc)
            accs.append(tta_acc)
            deltas.append(delta)
            err[c*n_training_series + n] = ( 100.0 *float(1.0 - len(data_per_serie[n+1])/len(data_per_serie[0])) ,  100.0 * (1.0 - float(tta_acc)))
            print(f'cycle {c} Error Rate is [{err}]')
#             inter_test = ClassificationInterpretation.from_learner(learn, dl =data_test)
#             inter_test.plot_confusion_matrix(figsize = (10,10))
            PATH='models/'+dump_1_features+'_'+str(c)+"_"+str(n)+".p"
            torch.save(model.state_dict(), PATH)
            csv = data_per_serie[n+1].to_csv('CSVz/'+dump_1_csvs+'_'+str(c)+"_"+str(n)+".csv", index = False)         

In [ ]:
# err = {}
print(err)

In [ ]:
for i in range(len(accs)):
    print(f'acc:{accs[i]}, delta:{deltas[i]}')

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(accs, label='Accuracy', color='tab:blue', marker='o')
plt.plot(deltas, label='Delta', color='tab:red', marker='o')
for i, (acc, delta) in enumerate(zip(accs, deltas)):
    plt.text(i, acc, f"{acc:.4f}", ha='center', va='bottom', fontsize=8, rotation=45, color='black', fontweight='bold')
    plt.text(i, delta, f"{delta:.4f}", ha='center', va='top', fontsize=8, rotation=45, color='black', fontweight='bold')
plt.title('Comparison of Accuracy and Delta Values', fontweight='bold', color = 'Black')
plt.xlabel('Training Series', fontweight='bold', color = 'black')
plt.ylabel('Delta Values and Accuracy (%)', fontweight='bold', color = 'Black')
plt.legend()
plt.grid(False)
plt.tight_layout()  # Adjust layout to prevent clipping of annotations
plt.show()

# Dump 0.50 with cleaning Scheme

In [ ]:
learn.load('baseline/'+best_pth)

In [ ]:
drop_idxx = []
los = []
top_losses = []
n_training_series = 15
min_epochs = 1
min_prob=0.5
max_prob=0.9
delta_epochs_per_serie =  0
min_lr = 1e-4
kappa_lr = 1 
k_samples = 0
data_per_serie = {}
data_per_serie[0] = train_df
largest = True
from_scratch = False
hybrid = False
cleaning_cycles = 3 
delta_max = 0.50
accs = []
deltas = []
err = {}
acc = []
for c in range(cleaning_cycles):
    for n in range(n_training_series):
        
        delta =  delta_max * math.sin( math.pi * float(n+1)/(n_training_series) )
        loss_func = losses.DumpledSCELossFlat(num_classes=196, alpha = 0.1, beta = 1.0, delta=delta, reduction='mean')
        k_samples = 50
        print(f'+++++ Training serie: inverse pyramid {n} ++++++++')
        epochs_per_serie = min_epochs + n * delta_epochs_per_serie
        lr = min_lr * kappa_lr * (n_training_series - n)
        data_per_serie[n+1] = data_definition(data_per_serie[n],img_size,batch_tfms,model,best_pth,k_samples=k_samples, 
                                              min_prob = min_prob, max_prob=max_prob, largest = largest)
        print("************Round: ",n)
        print("************data per serie:******************")
        print("************length of data per serie:******************")
        print(len(data_per_serie[n+1]))
        databunch = ( ImageDataLoaders.from_df(data_per_serie[n+1], valid_pct = 0.2,bs=bs,label_col=1,
                                        shuffle_train=True,
                                        item_tfms=Resize(imgs),
                                        batch_tfms = batch_tfms))
        learn =(learner_type(databunch, model,metrics=[accuracy],loss_func=loss_func,
                            cbs=[ShowGraphCallback(),checkpoints]).to_fp16())
        learn.unfreeze() 
        learn.load(best_pth)
        learn.fit_one_cycle(epochs_per_serie, lr_max=slice(1e-6, 5e-5))
        if n%3== 0:
            log_preds, y = learn.tta(dl=data_test)
            print("************Round: ",n)
            tta_acc = accuracy(log_preds, y)
            print(f'TTA Accuracy:{tta_acc}')
            acc.append(tta_acc)
            accs.append(tta_acc)
            deltas.append(delta)
            err[c*n_training_series + n] = ( 100.0 *float(1.0 - len(data_per_serie[n+1])/len(data_per_serie[0])) ,  100.0 * (1.0 - float(tta_acc)))
            print(f'cycle {c} Error Rate is [{err}]')
            inter_test = ClassificationInterpretation.from_learner(learn, dl =data_test)
            inter_test.plot_confusion_matrix(figsize = (10,10))
            PATH='models/'+dump_2_features+'_'+str(c)+"_"+str(n)+".p"
            torch.save(model.state_dict(), PATH)
            csv = data_per_serie[n+1].to_csv('CSVz/'+dump_2_csvs+'_'+str(c)+"_"+str(n)+".csv", index = False)         

In [ ]:
# err = {}
print(err)

In [ ]:
for i in range(len(accs)):
    print(f'acc:{accs[i]}, delta:{deltas[i]}')

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(accs, label='Accuracy', color='tab:blue', marker='o')
plt.plot(deltas, label='Delta', color='tab:red', marker='o')
for i, (acc, delta) in enumerate(zip(accs, deltas)):
    plt.text(i, acc, f"{acc:.4f}", ha='center', va='bottom', fontsize=8, rotation=45, color='black', fontweight='bold')
    plt.text(i, delta, f"{delta:.4f}", ha='center', va='top', fontsize=8, rotation=45, color='black', fontweight='bold')

plt.title('Comparison of Accuracy and Delta Values', fontweight='bold', color = 'Black')
plt.xlabel('Training Series', fontweight='bold', color = 'black')
plt.ylabel('Delta Values and Accuracy (%)', fontweight='bold', color = 'Black')
plt.legend()
plt.grid(False)
plt.tight_layout()
plt.show()
